Set up

In [21]:
import psycopg2
from sqlalchemy import create_engine
import os
import pandas as pd
from datetime import datetime

In [2]:
# Set the working directory
current_dir = os.getcwd()
print('current_dir: ',current_dir)
parent_dir = os.path.dirname(current_dir)

os.chdir(parent_dir)

print('Current working directory:', os.getcwd())

current_dir:  c:\Users\Utente\Documents\GitHub\SE4GEO-Lab\db_helper
Current working directory: c:\Users\Utente\Documents\GitHub\SE4GEO-Lab


In [23]:
ip = '192.168.30.19'
ip = 'localhost'
file = 'bin.txt'
with open('code/'+file, 'r') as f:
    engine = create_engine('postgresql://postgres:'+f.read()+'@'+ip+':5432/se4g') 
con = engine.connect()
print('connected with ',ip)

connected with  localhost


In [22]:
# Connect to the database
conn = psycopg2.connect(
    host = ip,
    database = "se4g",
    user = "postgres",
    password = "carIs3198"
)
print('connected with ',ip)

connected with  localhost


In [16]:
def insert_data(table_name, df, conn, df_columns = ['station_code', 
                                                    'station_name', 
                                                    'station_altitude', 
                                                    'network_countrycode', 
                                                    'pollutant', 
                                                    'value_datetime_begin',
                                                    'value_datetime_end',
                                                    'value_datetime_updated',
                                                    'value_numeric']):
    cur = conn.cursor()

    # Iterate over the DataFrame rows and insert data row by row
    for _, row in df.iterrows():
        # Generate the SQL INSERT statement
        insert_statement = f"INSERT INTO {table_name} ({', '.join(df_columns)}) VALUES ({', '.join(['%s'] * len(df_columns))})"
        values = tuple(row[col] for col in df_columns)

        # Execute the INSERT statement
        cur.execute(insert_statement, values)

    
    # Commit the changes and close the cursor 
    conn.commit()
    cur.close()

In [15]:
# Update the final dataset
def update_DB(new_df, conn, table_name='se4g_pollution'):

    # Open the dataset
    
    query = f"SELECT * FROM {table_name}"
    df = pd.read_sql_query(query, conn)

    df['value_datetime_begin'] = pd.to_datetime(df['value_datetime_begin'])
    new_df['value_datetime_begin'] = pd.to_datetime(new_df['value_datetime_begin'])

    # Filter rows from new_df based on the datetime
    filtered_rows = new_df[new_df['value_datetime_begin'] > df['value_datetime_begin'].max()]

    if filtered_rows.empty:
        print("Nothing to update inside dataset ",table_name)

    elif not filtered_rows.empty:

        # Update the dataset by adding the filtered rows
        #filtered_rows.to_sql(table_name, engine, if_exists='append', index=False)

        # Update the dataset by adding the filtered rows
        insert_data(table_name, filtered_rows, conn)
        print("Dataset ",table_name," updated successfully")

        return filtered_rows

In [27]:
def update_dashboard_DB(new_rows, table_name = 'se4g_dashboard'):
    
    columns = ['pollutant', 'country', 'month_day', 'value_numeric_mean', 'value_datetime_begin']

    country = {'AD': 'Andorra', 'SE': 'Sweden', 'DE': 'Germany', 'CY': 'Undefined', 'BE': 'Belgium',
               'FI': 'Finland', 'ES': 'Spain', 'CZ': 'Czech Republic', 'BG': 'Bulgaria', 'BA': 'Bosnia and Herzegovina',
               'EE': 'Estonia', 'CH': 'Switzerland', 'AT': 'Austria', 'DK': 'Denmark'}
    
    # Convert 'value_datetime_end' to datetime objects and extract the day
    datetime_objects = new_rows['value_datetime_end'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S%z'))
    
    new_rows['month_day'] = datetime_objects.dt.strftime('%m%d')
    
    # Compute daily mean of 'value_numeric' for each 'pollutant' and 'network_countrycode'
    daily_mean = new_rows.groupby(['pollutant', 'network_countrycode', 'month_day'])['value_numeric'].mean().reset_index()
    
    # Merge the daily mean back to the original dataframe
    new_rows = new_rows.merge(daily_mean, on=['pollutant', 'network_countrycode', 'month_day'], suffixes=('', '_mean'))
    
    new_rows['country'] = new_rows['network_countrycode'].map(country)
    new_rows = new_rows[columns].copy()
    
    new_rows = new_rows.drop_duplicates().reset_index(drop=True)
    new_rows = new_rows.sort_values('month_day')
    
    # Append the new_rows to the existing table
    new_rows.to_sql(table_name, engine, if_exists='append', index=False)

    if new_rows.empty:
        print("Nothing to update inside dataset ",table_name)

    elif not new_rows.empty:

        # Update the dataset by adding the filtered rows
        #filtered_rows.to_sql(table_name, engine, if_exists='append', index=False)

        # Update the dataset by adding the filtered rows
        insert_data(table_name, new_rows, conn, df_columns=columns)
        print("Dataset ",table_name," updated successfully")

# ADD CHECK -> NOT TO ADD THE SAME DATA

In [13]:
# Create a cursor
cur = conn.cursor()

query = "SELECT * FROM se4g_pollution"

# Execute the query
cur.execute(query)

# Fetch all the results
results = cur.fetchall()

# Get the column names from the cursor description
columns = [desc[0] for desc in cur.description]

# Create a DataFrame from the results and column names
df = pd.DataFrame(results, columns=columns)

# Close the cursor and connection
cur.close()

# Print the DataFrame
print(df.head())

  station_code        station_name  station_altitude network_countrycode  \
0      AD0942A  Escaldes-Engordany            1080.0                  AD   
1      AD0942A  Escaldes-Engordany            1080.0                  AD   
2      AD0942A  Escaldes-Engordany            1080.0                  AD   
3      AD0942A  Escaldes-Engordany            1080.0                  AD   
4      AD0942A  Escaldes-Engordany            1080.0                  AD   

  pollutant       value_datetime_begin         value_datetime_end  \
0       SO2  2023-05-22 06:00:00+01:00  2023-05-22 07:00:00+01:00   
1       SO2  2023-05-22 07:00:00+01:00  2023-05-22 08:00:00+01:00   
2       SO2  2023-05-22 08:00:00+01:00  2023-05-22 09:00:00+01:00   
3       SO2  2023-05-22 09:00:00+01:00  2023-05-22 10:00:00+01:00   
4       SO2  2023-05-22 10:00:00+01:00  2023-05-22 11:00:00+01:00   

  value_datetime_updated  value_numeric  
0                 +01:00            0.9  
1                 +01:00            1.3  
2 

In [10]:
from se4g_helper import download_request, build_dataframe, update_dataset, update_dashboard_dataset
# Download and get the dataframe file name
dir = download_request(folder_out = 'data_prova')

# Build the dataframe with the required structure
df = build_dataframe(dir, folder_out = 'data_prova')

# Update the final dataset
new_df = update_dataset(df, folder_out = 'data_prova')
                

# Update the dashboard dataset 
update_dashboard_dataset(df, folder_out = 'data_prova')

-----------------------------------------------------------------------
01-06-2023_14_17_11 directory created
Downloading: http://discomap.eea.europa.eu/map/fme/latest/AD_SO2.csv
Saved locally as: data_prova\01-06-2023_14_17_11\AD_SO2.csv 
-----
Downloading: http://discomap.eea.europa.eu/map/fme/latest/AD_NO.csv
Saved locally as: data_prova\01-06-2023_14_17_11\AD_NO.csv 
-----
Downloading: http://discomap.eea.europa.eu/map/fme/latest/AD_NO2.csv
Saved locally as: data_prova\01-06-2023_14_17_11\AD_NO2.csv 
-----
Downloading: http://discomap.eea.europa.eu/map/fme/latest/AD_CO.csv
Saved locally as: data_prova\01-06-2023_14_17_11\AD_CO.csv 
-----
Downloading: http://discomap.eea.europa.eu/map/fme/latest/AD_PM10.csv
Saved locally as: data_prova\01-06-2023_14_17_11\AD_PM10.csv 
-----
Downloading: http://discomap.eea.europa.eu/map/fme/latest/AT_SO2.csv
Saved locally as: data_prova\01-06-2023_14_17_11\AT_SO2.csv 
-----
Downloading: http://discomap.eea.europa.eu/map/fme/latest/AT_NO.csv
Saved lo

In [17]:
filtered_rows = update_DB(new_df, conn, table_name = 'se4g_pollution')

C:\Users\Utente\AppData\Local\Temp\ipykernel_20916\1742173976.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


Dataset  se4g_pollution  updated successfully


In [29]:
update_dashboard_DB(filtered_rows, table_name = 'se4g_dashboard')

Dataset  se4g_dashboard  updated successfully
Dataset se4g_dashboard updated successfully


In [9]:
# Create a cursor
cur = conn.cursor()

# Specify the datetime value
datetime_value_begin = '2023-05-29 08:00:00+01:00'
datetime_value_end = '2023-05-29 12:00:00+01:00'

# Specify the network countrycode
network_countrycode = 'AD'

# Construct the SQL query with the conditions
query = f"SELECT pollutant, value_datetime_begin, value_datetime_end FROM se4g_pollution WHERE value_datetime_end > %s AND value_datetime_end < %s AND network_countrycode = %s"

# Execute the query
cur.execute(query, (datetime_value_begin, datetime_value_end, network_countrycode))

# Fetch all rows from the result set
rows = cur.fetchall()

# Get the column names
columns = [desc[0] for desc in cur.description]

# Create a pandas DataFrame from the rows and columns
df = pd.DataFrame(rows, columns=columns)

# Close the cursor
cur.close()

print(df.head())

   pollutant    value_datetime_begin         value_datetime_end
0        SO2  2023-05-29 07:00:00+00  2023-05-29 09:00:00+01:00
1        SO2  2023-05-29 08:00:00+00  2023-05-29 10:00:00+01:00
2        SO2  2023-05-29 09:00:00+00  2023-05-29 11:00:00+01:00
3         NO  2023-05-29 07:00:00+00  2023-05-29 09:00:00+01:00
4         NO  2023-05-29 08:00:00+00  2023-05-29 10:00:00+01:00
5         NO  2023-05-29 09:00:00+00  2023-05-29 11:00:00+01:00
6        NO2  2023-05-29 07:00:00+00  2023-05-29 09:00:00+01:00
7        NO2  2023-05-29 08:00:00+00  2023-05-29 10:00:00+01:00
8        NO2  2023-05-29 09:00:00+00  2023-05-29 11:00:00+01:00
9         CO  2023-05-29 07:00:00+00  2023-05-29 09:00:00+01:00
10        CO  2023-05-29 08:00:00+00  2023-05-29 10:00:00+01:00
11        CO  2023-05-29 09:00:00+00  2023-05-29 11:00:00+01:00
12      PM10  2023-05-29 07:00:00+00  2023-05-29 09:00:00+01:00
13      PM10  2023-05-29 08:00:00+00  2023-05-29 10:00:00+01:00
14      PM10  2023-05-29 09:00:00+00  20

In [ ]:
conn.close()